In [15]:
import pandas as pd
from difflib import SequenceMatcher
import itertools

In [17]:
# import data into dataframes
zones = pd.read_csv(r"C:\Users\jmsav\root\git\savirame\oubt\taxi_zone_lookup.csv")
trips = pd.read_parquet(r"C:\Users\jmsav\root\git\savirame\oubt\yellow_tripdata_2025-08.parquet")

In [18]:
print(zones.head(10))
print(zones.info())
print(trips.info())
print(zones['Borough'].value_counts())

   LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone
5           6  Staten Island  Arrochar/Fort Wadsworth    Boro Zone
6           7         Queens                  Astoria    Boro Zone
7           8         Queens             Astoria Park    Boro Zone
8           9         Queens               Auburndale    Boro Zone
9          10         Queens             Baisley Park    Boro Zone
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       264 non-null  

In [19]:
# Analyze Zone Master Data
print(f"\n1. ZONE MASTER: {len(zones)} records")
print(f"   Columns: {list(zones.columns)}")


1. ZONE MASTER: 265 records
   Columns: ['LocationID', 'Borough', 'Zone', 'service_zone']


In [20]:
# Identify Reference Data
print(f"\n2. REFERENCE DATA:")
print(f"   - VendorIDs: {sorted(trips['VendorID'].unique())}")
print(f"   - RateCodes: {sorted(trips['RatecodeID'].dropna().unique())}")
print(f"   - PaymentTypes: {sorted(trips['payment_type'].unique())}")


2. REFERENCE DATA:
   - VendorIDs: [1, 2, 6, 7]
   - RateCodes: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 99.0]
   - PaymentTypes: [0, 1, 2, 3, 4]


In [21]:
# Transactional Data
print(f"\n3. TRANSACTIONAL DATA:")
print(f"   - Trip Records: {len(trips):,}")


3. TRANSACTIONAL DATA:
   - Trip Records: 3,574,091


In [7]:
# simple matching alogirthm to detect duplicates
'''
FOR each record A in dataset:
    FOR each record B in dataset:
        IF A.id ≠ B.id AND
           normalize(A.Zone) == normalize(B.Zone) AND
           normalize(A.Borough) == normalize(B.Borough):
        THEN
           mark A and B as duplicates

'''

'\nFOR each record A in dataset:\n    FOR each record B in dataset:\n        IF A.id ≠ B.id AND\n           normalize(A.Zone) == normalize(B.Zone) AND\n           normalize(A.Borough) == normalize(B.Borough):\n        THEN\n           mark A and B as duplicates\n\n'

In [22]:
# normalize text columns
# 1. Detect exact duplicates (Zone + Borough)
exact_duplicates = zones[
    zones.duplicated(subset=['Zone', 'Borough'], keep=False)
]

print("--- DUPLICATE RECORDS DETECTED ---")
print(exact_duplicates.sort_values(by='Zone'))

# 2. Create Golden Record (keep first occurrence)
zones_master_clean = zones.drop_duplicates(
    subset=['Zone', 'Borough'],
    keep='first'
)

# 3. Save the cleaned master data
zones_master_clean.to_csv('cleaned_taxi_zones.csv', index=False)

print(f"\nOriginal count: {len(zones)}")
print(f"Cleaned count: {len(zones_master_clean)}")

--- DUPLICATE RECORDS DETECTED ---
     LocationID    Borough                                           Zone  \
55           56     Queens                                         Corona   
56           57     Queens                                         Corona   
102         103  Manhattan  Governor's Island/Ellis Island/Liberty Island   
103         104  Manhattan  Governor's Island/Ellis Island/Liberty Island   
104         105  Manhattan  Governor's Island/Ellis Island/Liberty Island   

    service_zone  
55     Boro Zone  
56     Boro Zone  
102  Yellow Zone  
103  Yellow Zone  
104  Yellow Zone  

Original count: 265
Cleaned count: 262


In [23]:
# 4. Fuzzy Match: Compare all unique zone names
unique_zones = zones_master_clean['Zone'].dropna().unique().tolist()
fuzzy_matches = []

for z1, z2 in itertools.combinations(unique_zones, 2):
    score = SequenceMatcher(None, z1, z2).ratio()
    # 0.85 is a high similarity threshold
    if 0.85 <= score < 1.0:
        fuzzy_matches.append((z1, z2, round(score, 3)))

# Output results
print("Exact Duplicates Found:", len(exact_duplicates))
print("Top Fuzzy Matches:", fuzzy_matches[:5])

Exact Duplicates Found: 5
Top Fuzzy Matches: [('Bensonhurst East', 'Bensonhurst West', 0.875), ('Bushwick North', 'Bushwick South', 0.857), ('Crown Heights North', 'Crown Heights South', 0.895), ('East Harlem North', 'East Harlem South', 0.882), ('Financial District North', 'Financial District South', 0.917)]


In [24]:
# 1. Load the "Golden" Master Data
master_df = zones_master_clean
master_ids = set(master_df['LocationID'].unique())

# 2. Load the Transaction Data (Parquet)
# We only need the Location columns to save memory
transactions = trips[['PULocationID', 'DOLocationID']].copy()


# 3. Extract unique IDs from Transactions
pickup_ids = set(transactions['PULocationID'].unique())
dropoff_ids = set(transactions['DOLocationID'].unique())
all_transaction_ids = pickup_ids.union(dropoff_ids)

# 4. FIND THE ORPHANS (In Transactions but NOT in Master)
orphans = all_transaction_ids - master_ids

# 5. FIND UNUSED MASTER DATA (In Master but NOT in Transactions)
unused = master_ids - all_transaction_ids

print("--- REFERENTIAL INTEGRITY REPORT ---")
if not orphans:
    print("✅ SUCCESS: No orphaned records found. All trips map to valid zones.")
else:
    print(f"❌ ALERT: Found {len(orphans)} orphaned LocationIDs!")
    print(f"Orphaned IDs: {orphans}")
    
print(f"\nTotal Unique Zones in Transactions: {len(all_transaction_ids)}")
print(f"Total Golden Records in Master: {len(master_ids)}")
print(f"Unused Master Zones (No trips recorded): {len(unused)}")

--- REFERENTIAL INTEGRITY REPORT ---
❌ ALERT: Found 1 orphaned LocationIDs!
Orphaned IDs: {57}

Total Unique Zones in Transactions: 261
Total Golden Records in Master: 262
Unused Master Zones (No trips recorded): 2


In [25]:
# ============================================
# PREPARE MASTER DATA DataFrames FOR RDS
# ============================================

# 1. BOROUGHS - Extract unique boroughs
unique_boroughs = sorted(zones_master_clean['Borough'].dropna().unique())
boroughs_df = pd.DataFrame({
    'borough_name': unique_boroughs
})
print("boroughs_df:")
print(boroughs_df)

# 2. SERVICE ZONES - Extract unique service zones
unique_service_zones = sorted(zones_master_clean['service_zone'].dropna().unique())
service_zones_df = pd.DataFrame({
    'service_zone_name': unique_service_zones
})
print("\nservice_zones_df:")
print(service_zones_df)

# 3. ZONES (Golden Records) - Main master table
zones_for_db = zones_master_clean.rename(columns={
    'LocationID': 'location_id',
    'Zone': 'zone_name',
    'Borough': 'borough',
    'service_zone': 'service_zone'
})
zones_for_db = zones_for_db[['location_id', 'zone_name', 'borough', 'service_zone']]

print("\nzones_for_db (Golden Records):")
print(zones_for_db.head(10))
print(f"\nTotal: {len(zones_for_db)} golden records")

boroughs_df:
    borough_name
0          Bronx
1       Brooklyn
2            EWR
3      Manhattan
4         Queens
5  Staten Island
6        Unknown

service_zones_df:
  service_zone_name
0          Airports
1         Boro Zone
2               EWR
3       Yellow Zone

zones_for_db (Golden Records):
   location_id                zone_name        borough service_zone
0            1           Newark Airport            EWR          EWR
1            2              Jamaica Bay         Queens    Boro Zone
2            3  Allerton/Pelham Gardens          Bronx    Boro Zone
3            4            Alphabet City      Manhattan  Yellow Zone
4            5            Arden Heights  Staten Island    Boro Zone
5            6  Arrochar/Fort Wadsworth  Staten Island    Boro Zone
6            7                  Astoria         Queens    Boro Zone
7            8             Astoria Park         Queens    Boro Zone
8            9               Auburndale         Queens    Boro Zone
9           10      

In [26]:
# ============================================
# SIMPLIFIED: Build Master Data Tables in RDS
# (Auto-creates tables from DataFrame structure)
# ============================================

from sqlalchemy import create_engine, text

# RDS Configuration - Replace with your values
RDS_CONFIG = {
    'host': 'mdm-postgres.cqre4gyo8nft.us-east-1.rds.amazonaws.com',  # Fixed: added closing quote
    'port': 5432,
    'database': 'postgres',
    'user': 'postgres',       # TODO: Add your username
    'password':'savitha1'
}

# Build connection string
CONNECTION_STRING = (
    f"postgresql://{RDS_CONFIG['user']}:{RDS_CONFIG['password']}"
    f"@{RDS_CONFIG['host']}:{RDS_CONFIG['port']}/{RDS_CONFIG['database']}"
)

def build_mdm_tables_in_rds():
    """
    Build all master data tables in RDS PostgreSQL.
    Tables are auto-created from DataFrame structure - no manual DDL needed!
    """
    
    # 1. Create engine
    engine = create_engine(CONNECTION_STRING)
    
    # 2. Test connection
    try:
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        print("✅ Connected to RDS PostgreSQL")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None
    
    # 3. Create schema (only thing we need DDL for)
    with engine.connect() as conn:
        conn.execute(text("CREATE SCHEMA IF NOT EXISTS mdm"))
        conn.commit()
    print("✅ Schema 'mdm' created")
    
    # 4. Load all tables - to_sql auto-creates tables from DataFrame!
    tables = {
        'boroughs': boroughs_df,
        'service_zones': service_zones_df,
        'zones': zones_for_db  # Golden records
    }
    
    for table_name, df in tables.items():
        df.to_sql(
            name=table_name,
            con=engine,
            schema='mdm',
            if_exists='replace',  # Drops & recreates table automatically
            index=False
        )
        print(f"  ✅ mdm.{table_name}: {len(df)} records")
    
    # 5. Verify
    print("\n📊 Table Record Counts:")
    for table_name in tables.keys():
        count = pd.read_sql(f"SELECT COUNT(*) FROM mdm.{table_name}", engine).iloc[0, 0]
        print(f"   mdm.{table_name}: {count}")
    
    print("\n🎉 All master data tables created successfully!")
    return engine

# Execute
engine = build_mdm_tables_in_rds()  # Uncomment when RDS is configured

✅ Connected to RDS PostgreSQL
✅ Schema 'mdm' created
  ✅ mdm.boroughs: 7 records
  ✅ mdm.service_zones: 4 records
  ✅ mdm.zones: 262 records

📊 Table Record Counts:
   mdm.boroughs: 7
   mdm.service_zones: 4
   mdm.zones: 262

🎉 All master data tables created successfully!


In [27]:
df = pd.read_sql(
    "SELECT * FROM mdm.zones LIMIT 5",
    engine
)
print(df)


   location_id                zone_name        borough service_zone
0            1           Newark Airport            EWR          EWR
1            2              Jamaica Bay         Queens    Boro Zone
2            3  Allerton/Pelham Gardens          Bronx    Boro Zone
3            4            Alphabet City      Manhattan  Yellow Zone
4            5            Arden Heights  Staten Island    Boro Zone
